# APIM ❤️ AI Agents

## Model Context Protocol (MCP) lab
![flow](../../images/model-context-protocol.gif)

Playground to experiment the [Model Context Protocol](https://modelcontextprotocol.io/) with Azure API Management to enable plug & play of tools to LLMs. Leverages the [credential manager](https://learn.microsoft.com/en-us/azure/api-management/credentials-overview) for  managing OAuth 2.0 tokens to backend tools and [client token validation](https://learn.microsoft.com/en-us/azure/api-management/validate-jwt-policy) to ensure end-to-end authentication and authorization.   
This lab includes the following MCP servers:
- Basic oncall service: provides a tool to get a list of random people currently on-call with their status and time zone.
- Basic weather service: provide tools to get cities for a given country and retrieve random weather information for a specified city.
- GitHub Issues MCP Server: provide tools to authenticate on GitHub using the APIM Credential Manager, retrieves user information, and lists issues for a specified repository. This [sequence diagram](./diagrams/diagrams.md) explains the flow. 
- ServiceNow incidents MCP Server: provides tools to authenticates on ServiceNow using the APIM Credential Manager, lists incidents, retrieves a particular incident and create a new one.

### Prerequisites

- [Python 3.12 or later version](https://www.python.org/) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Python environment](https://code.visualstudio.com/docs/python/environments#_creating-environments) with the [requirements.txt](../../requirements.txt) or run `pip install -r requirements.txt` in your terminal
- [An Azure Subscription](https://azure.microsoft.com/free/) with [Contributor](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#contributor) + [RBAC Administrator](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#role-based-access-control-administrator) or [Owner](https://learn.microsoft.com/en-us/azure/role-based-access-control/built-in-roles/privileged#owner) roles
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed and [Signed into your Azure subscription](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

▶️ Click `Run All` to execute all steps sequentially, or execute them `Step by Step`...


<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 

In [12]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"

aiservices_config = [{"name": "foundry1", "location": "eastus2"}]

models_config = [{"name": "gpt-4o-mini", "publisher": "OpenAI", "version": "2024-07-18", "sku": "GlobalStandard", "capacity": 100}]

apim_sku = 'Basicv2'
apim_subscriptions_config = [{"name": "subscription1", "displayName": "Subscription 1"}, 
                             {"name": "subscription2", "displayName": "Subscription 2"}, 
                             {"name": "subscription3", "displayName": "Subscription 3"}]

inference_api_path = ""  # path to the inference API in the APIM service - use blank to use the default path
inference_api_type = "AzureOpenAI"  # options: AzureOpenAI, AzureAI, OpenAI, PassThrough
inference_api_version = "2025-03-01-preview"
foundry_project_name = deployment_name

build = 1
weather_mcp_server_image = "weather-mcp-server"
weather_mcp_server_src = "../../shared/mcp-servers/weather/http"

oncall_mcp_server_image = "oncall-mcp-server"
oncall_mcp_server_src = "../../shared/mcp-servers/oncall/http"

github_mcp_server_image = "github-mcp-server"
github_mcp_server_src = "../../shared/mcp-servers/github/http"

utils.print_ok('Notebook initialized')

✅ Notebook initialized ⌚ 10:37:04.580467 


In [13]:
# Ensure you have the latest version of the Azure AI Projects package
# This is required for the MCP client to work properly with the Azure AI Foundry service if you want to run the AI Agent Service demo
%pip install azure-ai-projects==1.0.0b12 agent-framework

Note: you may need to restart the kernel to use updated packages.


<a id='1'></a>
### 1️⃣ Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [14]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

⚙️ Running: az account show 
✅ Retrieved az account ⌚ 10:37:15.427472 :3s]
👉🏽 Current user: lproux@microsoft.com
👉🏽 Tenant ID: 2b9d9f47-1fb6-400a-a438-39fe7d768649
👉🏽 Subscription ID: d334f2cd-3efd-494e-9fd3-2470b1a13e4c


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [15]:
# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "aiServicesConfig": { "value": aiservices_config },
        "modelsConfig": { "value": models_config },
        "apimSubscriptionsConfig": { "value": apim_subscriptions_config },
        "inferenceAPIPath": { "value": inference_api_path },
        "inferenceAPIType": { "value": inference_api_type },
        "foundryProjectName": { "value": foundry_project_name },
    }
}

# Write the parameters to the params.json file
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters params.json",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

⚙️ Running: az group show --name lab-model-context-protocol 
👉🏽 Using existing resource group 'lab-model-context-protocol'
⚙️ Running: az deployment group create --name model-context-protocol --resource-group lab-model-context-protocol --template-file main.bicep --parameters params.json 
✅ Deployment 'model-context-protocol' succeeded ⌚ 10:40:08.485204 :48s]


<a id='3'></a>
### 3️⃣ Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [16]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    ai_foundry_project_endpoint = utils.get_deployment_output(output, 'foundryProjectEndpoint', 'AI Foundry Project Endpoint')
    log_analytics_id = utils.get_deployment_output(output, 'logAnalyticsWorkspaceId', 'Log Analytics Id')
    apim_resource_name = utils.get_deployment_output(output, 'apimResourceName', 'APIM Resource Name')
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscriptions = json.loads(utils.get_deployment_output(output, 'apimSubscriptions').replace("\'", "\""))
    for subscription in apim_subscriptions:
        subscription_name = subscription['name']
        subscription_key = subscription['key']
        utils.print_info(f"Subscription Name: {subscription_name}")
        utils.print_info(f"Subscription Key: ****{subscription_key[-4:]}")
    api_key = apim_subscriptions[0].get("key") # default api key to the first subscription key
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')
    container_registry_name = utils.get_deployment_output(output, 'containerRegistryName', 'Container Registry Name')
    weather_containerapp_resource_name = utils.get_deployment_output(output, 'weatherMCPServerContainerAppResourceName', 'Weather Container App Resource Name')
    oncall_containerapp_resource_name = utils.get_deployment_output(output, 'oncallMCPServerContainerAppResourceName', 'Oncall Container App Resource Name')
    github_containerapp_resource_name = utils.get_deployment_output(output, 'gitHubMCPServerContainerAppResourceName', 'GitHub Container App Resource Name')



⚙️ Running: az deployment group show --name model-context-protocol -g lab-model-context-protocol 
✅ Retrieved deployment: model-context-protocol ⌚ 10:40:13.573455 :5s]
👉🏽 AI Foundry Project Endpoint: https://foundry1-3ilfnkpe2evck.services.ai.azure.com/api/projects/model-context-protocol-foundry1
👉🏽 Log Analytics Id: 2c912e65-0d89-4116-9fda-00191d72d81c
👉🏽 APIM Resource Name: apim-3ilfnkpe2evck
👉🏽 APIM Service Id: /subscriptions/d334f2cd-3efd-494e-9fd3-2470b1a13e4c/resourceGroups/lab-model-context-protocol/providers/Microsoft.ApiManagement/service/apim-3ilfnkpe2evck
👉🏽 APIM API Gateway URL: https://apim-3ilfnkpe2evck.azure-api.net
👉🏽 Subscription Name: subscription1
👉🏽 Subscription Key: ****a052
👉🏽 Subscription Name: subscription2
👉🏽 Subscription Key: ****7658
👉🏽 Subscription Name: subscription3
👉🏽 Subscription Key: ****48be
👉🏽 Application Insights Name: insights-3ilfnkpe2evck
👉🏽 Container Registry Name: acr3ilfnkpe2evck
👉🏽 Weather Container App Resource Name: aca-weather-3ilfnkpe2evck

<a id='4'></a>
### 4️⃣ Build and deploy the MCP Servers



In [17]:
build = build + 1 # increment the build number

utils.run(f"az acr build --image {weather_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {weather_mcp_server_src}/Dockerfile {weather_mcp_server_src}/. --no-logs", 
          "Weather MCP Server image was successfully built", "Failed to build the Weather MCP Server image")
utils.run(f'az containerapp update -n {weather_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{weather_mcp_server_image}:v0.{build}"', 
          "Weather MCP Server deployment succeeded", "Weather MCP Server deployment failed")

utils.run(f"az acr build --image {oncall_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {oncall_mcp_server_src}/Dockerfile {oncall_mcp_server_src}/. --no-logs", 
          "Oncall MCP Server image was successfully built", "Failed to build the Oncall MCP Server image")
utils.run(f'az containerapp update -n {oncall_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{oncall_mcp_server_image}:v0.{build}"', 
          "Oncall MCP Server deployment succeeded", "Oncall MCP Server deployment failed")

utils.run(f"az acr build --image {github_mcp_server_image}:v0.{build} --resource-group {resource_group_name} --registry {container_registry_name} --file {github_mcp_server_src}/Dockerfile {github_mcp_server_src}/. --no-logs", 
          "GitHub MCP Server image was successfully built", "Failed to build the GitHub MCP Server image")
utils.run(f'az containerapp update -n {github_containerapp_resource_name} -g {resource_group_name} --image "{container_registry_name}.azurecr.io/{github_mcp_server_image}:v0.{build}"', 
          "GitHub MCP Server deployment succeeded", "GitHub MCP Server deployment failed")

⚙️ Running: az acr build --image weather-mcp-server:v0.2 --resource-group lab-model-context-protocol --registry acr3ilfnkpe2evck --file ../../shared/mcp-servers/weather/http/Dockerfile ../../shared/mcp-servers/weather/http/. --no-logs 
✅ Weather MCP Server image was successfully built ⌚ 10:41:24.435224 :10s]
⚙️ Running: az containerapp update -n aca-weather-3ilfnkpe2evck -g lab-model-context-protocol --image "acr3ilfnkpe2evck.azurecr.io/weather-mcp-server:v0.2" 
✅ Weather MCP Server deployment succeeded ⌚ 10:41:50.813990 :26s]
⚙️ Running: az acr build --image oncall-mcp-server:v0.2 --resource-group lab-model-context-protocol --registry acr3ilfnkpe2evck --file ../../shared/mcp-servers/oncall/http/Dockerfile ../../shared/mcp-servers/oncall/http/. --no-logs 
✅ Oncall MCP Server image was successfully built ⌚ 10:43:02.784150 :11s]
⚙️ Running: az containerapp update -n aca-oncall-3ilfnkpe2evck -g lab-model-context-protocol --image "acr3ilfnkpe2evck.azurecr.io/oncall-mcp-server:v0.2" 
✅ Onca

<a id='testconnection'></a>
### 🧪 Test the connection to the MCP servers and List Tools

💡 To integrate MCP servers in VS Code, use the MCP server URL  `../mcp ` for configuration in GitHub Copilot Agent Mode

If the notebook is run again, the JWT validation that gets applied to the policies later on must first be removed. Otherwise, the next calls below will fail as auth is not yet expected to be in place.

In [18]:
policy_xml_file = "src/github/apim-api/no-auth-client-policy.xml"

with open(policy_xml_file, 'r') as file:
    policy_xml = file.read()
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp-tools", "mcp", policy_xml)

⚙️ Running: az account get-access-token --resource https://management.azure.com/ 
✅ Successfully obtained access token ⌚ 10:45:26.534939 :21s]
Updating the API policy...
Response status: 400 - Bad Request


In [19]:
import os, json, asyncio, time, requests
from mcp import ClientSession
from mcp.client.sse import sse_client
from mcp.client.streamable_http import streamablehttp_client
import nest_asyncio
nest_asyncio.apply()

async def list_tools(server_url, authorization_header = None):
    headers = {"Authorization": authorization_header} if authorization_header else None
    streams = None
    session = None
    tools = []
    try:
        streams_ctx = streamablehttp_client(server_url, headers)
        streams = await streams_ctx.__aenter__()
        session_ctx = ClientSession(streams[0], streams[1])
        session = await session_ctx.__aenter__()
        await session.initialize()
        response = await session.list_tools()
        tools = response.tools
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        # Ensure session and streams are closed if they were opened
        if session is not None:
            await session_ctx.__aexit__(None, None, None) # type: ignore
        if streams is not None:
            await streams_ctx.__aexit__(None, None, None) # type: ignore
    if tools:
        print(f"✅ Connected to server {server_url}")
        print("⚙️ Tools:")
        for tool in tools:
            print(f"  - {tool.name}")
            print(f"     Input Schema: {tool.inputSchema}")

try:    
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/weather"))
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/oncall"))
    asyncio.run(list_tools(f"{apim_resource_gateway_url}/github"))

finally:
    print(f"✅ Connection closed")


✅ Connected to server https://apim-3ilfnkpe2evck.azure-api.net/weather
⚙️ Tools:
  - get_cities
     Input Schema: {'properties': {'country': {'type': 'string'}}, 'required': ['country'], 'type': 'object'}
  - get_weather
     Input Schema: {'properties': {'city': {'type': 'string'}}, 'required': ['city'], 'type': 'object'}
✅ Connected to server https://apim-3ilfnkpe2evck.azure-api.net/oncall
⚙️ Tools:
  - get_oncall_list
     Input Schema: {'properties': {}, 'type': 'object'}
✅ Connected to server https://apim-3ilfnkpe2evck.azure-api.net/github
⚙️ Tools:
  - authorize_github
     Input Schema: {'properties': {}, 'type': 'object'}
  - get_user
     Input Schema: {'properties': {}, 'type': 'object'}
  - get_issues
     Input Schema: {'properties': {'username': {'type': 'string'}, 'repo': {'type': 'string'}}, 'required': ['username', 'repo'], 'type': 'object'}
✅ Connection closed


<a id='inspector'></a>
### 🧪 (optional) Use the [MCP Inspector](https://modelcontextprotocol.io/docs/tools/inspector) for testing and debugging the MCP servers

#### Execute the following steps:
1. Execute `npx @modelcontextprotocol/inspector` in a terminal
2. Open the provided URL in a browser
3. Set the transport type as SSE
4. Provide the MCP server url and click connect
5. Select the "Tools" tab to see and run the available tools

<a id='functioncalling'></a>
### 🧪 Run an OpenAI completion with MCP tools

👉 Both the calls to Azure OpenAI and the MCP tools will be managed through Azure API Management.  


In [20]:
# type: ignore
import json, asyncio
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client
from openai import AzureOpenAI
import nest_asyncio
nest_asyncio.apply()

async def call_tool(mcp_session, function_name, function_args):
    try:
        func_response = await mcp_session.call_tool(function_name, function_args)
        func_response_content = func_response.content
    except Exception as e:
        func_response_content = json.dumps({"error": str(e)})
    return str(func_response_content)

async def run_completion_with_tools(server_url, prompt):
    streams = None
    session = None
    try:
        streams_ctx = streamablehttp_client(server_url)
        streams = await streams_ctx.__aenter__()
        session_ctx = ClientSession(streams[0], streams[1])
        session = await session_ctx.__aenter__()
        await session.initialize()
        response = await session.list_tools()
        tools = response.tools
        print(f"✅ Connected to server {server_url}")
        openai_tools = [{
                "type": "function",
                "function": {
                    "name": tool.name,
                    "parameters": tool.inputSchema
                },
            } for tool in tools]

        # Step 1: send the conversation and available functions to the model
        print("▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt")
        client = AzureOpenAI(
            azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
            api_key=api_key,
            api_version=inference_api_version
        )            
        messages = [ {"role": "user", "content": prompt} ]
        response = client.chat.completions.create(
            model=models_config[0]['name'],
            messages=messages,
            tools=openai_tools,
        )
        response_message = response.choices[0].message
        tool_calls = response_message.tool_calls
        if tool_calls:
            # Step 2: call the function
            messages.append(response_message)  # extend conversation with assistant's reply
            # send the info for each function call and function response to the model
            print("▶️ Step 2: call the functions")
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                print(f"   Function Name: {function_name} Function Args: {function_args}")

                function_response = await call_tool(session, function_name, function_args)
                # Add the tool response
                print(f"   Function response: {function_response}")
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )  # extend conversation with function response
            print("▶️ Step 3: finish with a completion to anwser the user prompt using the function response")
            second_response = client.chat.completions.create(
                model=models_config[0]['name'],
                messages=messages,
            )  # get a new response from the model where it can see the function response
            print("💬", second_response.choices[0].message.content)
    except Exception as e:
        print(f"❌ Error: {e}")
    finally:
        if session is not None:
            await session_ctx.__aexit__(None, None, None)
        if streams is not None:
            await streams_ctx.__aexit__(None, None, None)

asyncio.run(run_completion_with_tools(f"{apim_resource_gateway_url}/weather", "What's the current weather in Lisbon?"))
asyncio.run(run_completion_with_tools(f"{apim_resource_gateway_url}/oncall", "Who's oncall in CET today?"))


✅ Connected to server https://apim-3ilfnkpe2evck.azure-api.net/weather
▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt
▶️ Step 2: call the functions
   Function Name: get_weather Function Args: {'city': 'Lisbon'}
   Function response: [TextContent(type='text', text="{'city': 'Lisbon', 'condition': 'Snowy', 'temperature': 24.15, 'humidity': 23.78}", annotations=None, meta=None)]
▶️ Step 3: finish with a completion to anwser the user prompt using the function response
💬 The current weather in Lisbon is snowy, with a temperature of 24.15°C and a humidity level of 23.78%.
✅ Connected to server https://apim-3ilfnkpe2evck.azure-api.net/oncall
▶️ Step 1: start a completion to identify the appropriate functions to invoke based on the prompt
▶️ Step 2: call the functions
   Function Name: get_oncall_list Function Args: {}
   Function response: [TextContent(type='text', text="[{'id': 1, 'firstName': 'Julia', 'lastName': 'Smith', 'alias': 'jsmith'

<a id='Microsoft Agent Framework'></a>
### 🧪 Execute an [Agent Framework Agent using MCP Tools](https://learn.microsoft.com/en-us/agent-framework/user-guide/overview) via Azure API Management

In [21]:
import asyncio

from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import MCPStreamableHTTPTool
from pydantic import Field

async def non_streaming_example() -> None:
    """Example of non-streaming response (get the complete result at once)."""
    print("=== Non-streaming Response Example ===")

    async with (
        MCPStreamableHTTPTool(
            name="weather",
            url=f"{apim_resource_gateway_url}/weather",
            headers={"Authorization": "Bearer token"},
            description="Weather Plugin",
        )
    ) as mcp_tool:
        chatClient = AzureOpenAIChatClient(
            deployment_name=models_config[0]['name'],
            endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
            api_key=api_key,
            api_version=inference_api_version,
        )
        agent = chatClient.create_agent(
            instructions="You are a helpful weather agent.",
            tools=[mcp_tool],
            # deployment_name=models_config[0]['name'],
        )

        query = "What's the weather like in Seattle?"
        print(f"User: {query}")
        result = await agent.run(query)
        print(f"Result: {result}\n")


async def main() -> None:
    print("=== Basic Azure Chat Client Agent Example ===")
    await non_streaming_example()


if __name__ == "__main__":
    asyncio.run(main())

=== Basic Azure Chat Client Agent Example ===
=== Non-streaming Response Example ===
User: What's the weather like in Seattle?
Result: The weather in Seattle is currently rainy, with a temperature of 20.38°C and humidity at 26.44%.



<a id='Azure AI Agents'></a>
### 🧪 Execute an [Azure AI Foundry Agent using MCP Tools](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol) via Azure API Management

In [22]:
from azure.ai.agents.models import ListSortOrder, MessageTextContent, McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient(endpoint=ai_foundry_project_endpoint,
            credential=DefaultAzureCredential())
agents_client = project_client.agents

# MCP tool definition
mcp_tool = McpTool(
    server_label="weather",
    server_url=f"{apim_resource_gateway_url}/weather",
    #allowed_tools=[],          # Optional initial allow‑list
)

#mcp_tool.update_headers({"Authorization": f"Bearer {apim_subscription_key}"})

prompt = "What's the weather in Lisbon, Cairo and London?"

# Agent creation
agent = agents_client.create_agent(
    model=str(models_config[0].get('name')),
    name="agent-mcp",
    instructions="You are a sarcastic AI agent. Use the tools provided to answer the user's questions. Be sure to cite your sources and answer in details.",
    tools=mcp_tool.definitions
)
print(f"🎉 Created agent, agent ID: {agent.id}")
print(f"✨ MCP Server: {mcp_tool.server_label} at {mcp_tool.server_url}")

# Thread creation
thread = agents_client.threads.create()
print(f"🧵 Created thread, thread ID: {thread.id}")

# Message creation
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=prompt,
)
print(f"💬 Created message, message ID: {message.id}")

mcp_tool.set_approval_mode("never")          # Disable human approval

# Run
run = agents_client.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(2)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)
    print(f"⏳ Run status: {run.status}")
if run.status == "failed":
    print(f"❌ Run error: {run.last_error}")

# Get Run steps
run_steps = agents_client.run_steps.list(thread_id=thread.id, run_id=run.id)
print()

for step in run_steps:
    print(f"🔄 Run step: {step.id}, status: {step.status}, type: {step.type}")
    if step.type == "tool_calls":
        print(f"🛠️ Tool call details:")
        for tool_call in step.step_details.tool_calls:
            print(json.dumps(tool_call.as_dict(), indent=5))

# Get the messages in the thread
print("\n📜 Messages in the thread:")
messages = agents_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

for item in messages:
    last_message_content = item.content[-1]
    if isinstance(last_message_content, MessageTextContent):
        print(f"🗨️ {item.role}: {last_message_content.text.value}")

# Clean up resources
# agents_client.delete_agent(agent.id)


🎉 Created agent, agent ID: asst_ThLQTEaslyUFYiTdE1o08U5y
✨ MCP Server: weather at https://apim-3ilfnkpe2evck.azure-api.net/weather
🧵 Created thread, thread ID: thread_pQthOmOsyb29UelXtPwklDgP
💬 Created message, message ID: msg_U6dsdub37dYC4IY2QgZxAcfd
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.IN_PROGRESS
⏳ Run status: RunStatus.COMPLETED

🔄 Run step: step_2drtQ7GtrJ38t2CDiNFXAOUE, status: RunStepStatus.COMPLETED, type: RunStepType.MESSAGE_CREATION
🔄 Run step: step_NGazDlphlS4bHd6uIix7SXTJ, status: RunStepStatus.COMPLETED, type: RunStepType.TOOL_CALLS
🛠️ Tool call details:
{
     "id": "call_OSEohSgUE8mBPrOGmWem8Hfa",
     "type": "mcp",
     "arguments": "{\"city\":\"London\"}",
     "name": "get_weather",
     "server_label": "weather",
     "output": "{'city': 'London', 'condition': 'Cloudy', 'temperature': -0.05, 'humidity': 44.09}"
}
🔄 Run step: step_LsciCXGKXVZBYuM3iETCv0mg, status: RunStepStatus.COMPLETED, type: RunStepType.TOOL_CALLS
🛠️ Tool call details:
{
   

<a id='sk'></a>
### 🧪 Execute a [Semantic Kernel Agent using MCP Tools](https://devblogs.microsoft.com/semantic-kernel/integrating-model-context-protocol-tools-with-semantic-kernel-a-step-by-step-guide/) via Azure API Management

In [26]:
import asyncio
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStreamableHttpPlugin

user_input = "What's the current weather in Paris?"

async def main():
    # 1. Create the agent
    async with MCPStreamableHttpPlugin(
        name="Weather",
        url=f"{apim_resource_gateway_url}/weather",
        description="Weather Plugin",
    ) as weather_plugin:
        agent = ChatCompletionAgent(
            service=AzureChatCompletion(
                endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                api_key=api_key,
                api_version=inference_api_version,                
                deployment_name=models_config[0]['name']  # Use the first model from the models_config
            ),
            name="IssueAgent",
            instructions="Answer questions about the Weather.",
            plugins=[weather_plugin],
        )

        thread: ChatHistoryAgentThread | None = None

        print(f"# User: {user_input}")
        # 2. Invoke the agent for a response
        response = await agent.get_response(messages=user_input, thread=thread)
        print(f"# {response.name}: {response} ")
        thread = response.thread # type: ignore

        # 3. Cleanup: Clear the thread
        await thread.delete() if thread else None

if __name__ == "__main__":
    asyncio.run(main())

# User: What's the current weather in Paris?
# IssueAgent: The current weather in Paris is snowy, with a temperature of -9.7°C and humidity at 27.59%. 


<a id='autogen'></a>
### 🧪 Execute an [AutoGen Agent using MCP Tools](https://microsoft.github.io/autogen/stable/reference/python/autogen_ext.tools.mcp.html) via Azure API Management

In [24]:
import asyncio
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_ext.tools.mcp import mcp_server_tools, StreamableHttpServerParams
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken

async def run_agent(url, prompt) -> None:
    # Create server params for the remote MCP service
    server_params = StreamableHttpServerParams(
        url=url,
        headers={"Content-Type": "application/json"},
        timeout=30,  # Connection timeout in seconds
    )

    # Get all available tools
    tools = await mcp_server_tools(server_params)

    # Create an agent that can use the translation tool
    model_client = AzureOpenAIChatCompletionClient(azure_deployment=models_config[0]['name'], model=models_config[0]['name'],
                azure_endpoint=f"{apim_resource_gateway_url}/{inference_api_path}",
                api_key=api_key,
                api_version=inference_api_version
    )
    agent = AssistantAgent(
        name="weather",
        model_client=model_client,
        reflect_on_tool_use=True,
        tools=tools, # type: ignore
        system_message="You are a helpful assistant.",
    )
    await Console(
        agent.run_stream(task=prompt)
    )

asyncio.run(run_agent(f"{apim_resource_gateway_url}/weather", "What's the weather in Lisbon, Cairo and London?"))


---------- TextMessage (user) ----------
What's the weather in Lisbon, Cairo and London?
---------- ToolCallRequestEvent (weather) ----------
[FunctionCall(id='call_XR8cBG5QOBGnIBc8I6gcna4v', arguments='{"city": "Lisbon"}', name='get_weather'), FunctionCall(id='call_FIpFxv4sxRax5b1cjHNLUXCT', arguments='{"city": "Cairo"}', name='get_weather'), FunctionCall(id='call_AjnxFriWGFWXC8gqCOX6CKVX', arguments='{"city": "London"}', name='get_weather')]
---------- ToolCallExecutionEvent (weather) ----------
[FunctionExecutionResult(content='[{"type": "text", "text": "{\'city\': \'Lisbon\', \'condition\': \'Sunny\', \'temperature\': 14.25, \'humidity\': 56.53}", "annotations": null}]', name='get_weather', call_id='call_XR8cBG5QOBGnIBc8I6gcna4v', is_error=False), FunctionExecutionResult(content='[{"type": "text", "text": "{\'city\': \'Cairo\', \'condition\': \'Snowy\', \'temperature\': 15.87, \'humidity\': 75.2}", "annotations": null}]', name='get_weather', call_id='call_FIpFxv4sxRax5b1cjHNLUXCT',

<a id='githubconfig'></a>
### 5️⃣ Create a GitHub OAuth app and configure the credential provider

#### Step 1 - [Register the application in GitHub](https://learn.microsoft.com/en-us/azure/api-management/credentials-how-to-github#step-1-register-an-application-in-github)

👉 Use the Authorization callback URL that is provided below  
👉 Copy the Client ID and Client secret

#### Step 2 - [Configure the credential provider in API Management](https://learn.microsoft.com/en-us/azure/api-management/credentials-how-to-github#step-2-configure-a-credential-provider-in-api-management)

👉 You just need to update the Client ID and Client secret on the existing `github` credential manager provider  
👉 Disregard the remaining steps outlined in the documentation


<a id='servicenowconfig'></a>
### 6️⃣ Create a ServiceNow OAuth app and configure the credential provider

ℹ️ If you do not wish to use ServiceNow, please skip these steps

#### Step 1 - [Register the application in ServiceNow](https://www.servicenow.com/docs/bundle/yokohama-application-development/page/build/pipelines-and-deployments/task/create-oauth-api-endpoints-for-external-clients.html)

👉 Use the Authorization callback URL that is provided bellow  
👉 Copy the Client ID and Client secret

#### Step 2 - Configure the credential provider in API Management

👉 You just need to update the Client ID and Client secret on the existing `servicenonw` credential manager provider  

In [25]:
print(f"Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/{apim_resource_name}")

Authorization callback URL: https://authorization-manager.consent.azure-apim.net/redirect/apim/apim-3ilfnkpe2evck


<a id='githubtest'></a>
### 🧪 Run the GitHub MCP Server with VS Code to retrieve GitHub Issues

1. [Configure the GitHub MCP Server in VS Code](https://code.visualstudio.com/docs/copilot/chat/mcp-servers#_add-an-mcp-server) 
2. Type in the chat the following prompt: `Please list all the issues assigned to me in the GitHub repository {your-repo-name} under the organization {your-org-name}`
3. The agent will suggest running the `authorize_github` tool.
4. Once the user accepts to run the tool, the agent will call the `authorize_github` and provide an URL to proceed with the authentication and authorization on GitHub.
5. After the user confirms that it's done, the agent will suggest running the `get_user` tool.
6. Once the user accepts to run the `get_user` tool, the agent will call the tool, return user information as context and suggest running the `get_issues` tool.
7. Once the user accepts to run the `get_issues` tool, the agent will provide the list of issues from the given repo.

<a id='servicenowtest'></a>
### 🧪 Run the ServiceNow MCP Server with VS Code to manage ServiceNow incidents

1. [Configure the ServiceNow MCP Server in VS Code](https://code.visualstudio.com/docs/copilot/chat/mcp-servers#_add-an-mcp-server) 
2. Type in the chat the following prompt: `Please list my servicenow incidents`
3. The agent will suggest running the `authorize_servicenow` tool.
4. Once the user accepts to run the tool, the agent will call the `authorize_servicenow` and provide an URL to proceed with the authentication and authorization on ServiceNow.
5. After the user confirms that it's done, the agent will suggest running the `list_incidents` tool.
6. Once the user accepts to run the `list_incidents` tool, the agent will provide the list of incidents for the connected ServiceNow instance.
7. You can also retrieve details for a specific incident or create a new one.

✨ Type in the chat the following prompt: `Create a ServiceNow incident for each GitHub issue`. To combine GitHub and ServiceNow MCP Servers.

<a id='validate-jwt'></a>
### 🔐 (Optional) Implement [authorization policies](src/github/apim-api/auth-client-policy.xml) on MCP endpoints

👉 To ensure the enforcement of valid security tokens, we apply the `validate-jwt` policy to the `/sse` and `/messages` endpoints. The following code snippet demonstrates the application of this policy to GitHub API operations for token validation:

In [ ]:
policy_xml_file = "src/github/apim-api/auth-client-policy.xml"

with open(policy_xml_file, 'r') as file:
    policy_xml = file.read()
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "sse", policy_xml)
    utils.update_api_operation_policy(subscription_id, resource_group_name, apim_resource_name, "github-mcp", "messages", policy_xml)

<a id='unauthorizedtest'></a>
### 🧪 Test the authorization **WITHOUT** a valid token

In [ ]:
# Unauthenticated call should fail with 401 Unauthorized
import requests
utils.print_info("Calling sse endpoint WITHOUT authorization...")
response = requests.get(f"{apim_resource_gateway_url}/github/sse", headers={"Content-Type": "application/json"})
if response.status_code == 401:
    utils.print_ok("Received 401 Unauthorized as expected")
elif response.status_code == 200:
    utils.print_error("Call succeeded. Double check that validate-jwt policy has been deployed to sse endpoint")
else:
    utils.print_error(f"Unexpected status code: {response.status_code}")


<a id='authorizedtest'></a>
### 🧪 Test the authorization **WITH** a valid token

In [ ]:
import requests
# Authenticated call should succeed
utils.print_info("Calling sse endpoint WITH authorization...")
output = utils.run("az account get-access-token --resource \"https://azure-api.net/authorization-manager\"")
if output.success and output.json_data:
    access_token = output.json_data.get('accessToken')
    response = requests.get(f"{apim_resource_gateway_url}/github/sse", stream=True,
                            headers={"Content-Type": "application/json", "Authorization": "Bearer " + str(access_token)})
    if response.status_code == 200:
        utils.print_ok("Received status code 200 as expected")
    else:
        utils.print_error(f"Unexpected status code: {response.status_code}")



<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.